---
title: "Optimización de parámetros"
theme: Montpellier
author: "Máster en Ciencias de Datos e Ingeniería de Computadores, Minería de Datos - Preprocesamiento y clasificación"
date: 11/11/2024
date-format: long
toc: true
toc-title: Tabla de Contenidos
toc-depth: 1
execute:
  echo: true
output:
  beamer_presentation:
    slide_level: 1
format:
  html:
    code-fold: false
    code-summary: "Muestra código"
    fig-width: 5
    fig-height: 3
    fig-align: left
  beamer:
    fig-width: 4
    fig-height: 2
  revealjs:
    theme: dark
    fig-align: left
    fig-height: 5
    fig-cap-location: margin
    smaller: true
---

# Introducción

## Introducción

El comportamiento de los algoritmos de ML dependen de muchos parámetros.

- Los parámetros por defecto de Scikit-learn son muy buenos, no los mejores para un problema en concreto.

- Importante: Cuidado con el sobre-aprendizaje, validar siempre.

- Hacerlo _a mano_ es muy laborioso, pero Sklearn permite métodos para ajustar los parámetros a los métodos.

- Los modelos de _tuning_ _envuelven_ al clasificador, y modifican el método fit().

- Aún así es importante ajustar los más relevantes, no muchos.

- Scikit-learn siempre aplica los métodos con validación cruzada para evitar sobre-aprendizaje.

::: {.notes}
Sklearn proporciona métodos para buscar los parámetros óptimos de los métodos, ajustándolos **al conjunto de entrenamiento** actual. Para ello, aplica una validación cruzada interna al propio conjunto de entrenamiento que podrá ser el conjunto completo o, a su vez, la partición de entrenamiento de otro esquema de validación. Esta validación cruzada interna y temporal es utilizada para comprobar el efecto de los parámetros en el clasificador, de forma **exhaustiva**.
:::

# Técnica de Rejilla: Grid Search

## Técnica de Rejilla: Grid Search

Es la técnica exhaustiva.

Es fácil, para cada parámetro a optimizar se indica los valores.

- Cuidado, puede llevar mucho tiempo, calcular lo que tardará.

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report

breastCancer = datasets.load_breast_cancer()
X_b = breastCancer.data
y_b = breastCancer.target

## Ejemplo de GridSearchCV

In [ ]:
#| code-fold: true
from sklearn.svm import SVC

# No necesitamos especificar los parámetros
svc = SVC() 

# Estudiamos dos valores de kernel, y números 1 y 10 para C
parametros = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

# Aplica por defecto cv=5, lo pongo para mostrarlo
ajuste = GridSearchCV(svc, parametros, cv=5, verbose=3)

#La búsqueda de parámetros se hace con el método fit, como si fuera un clasificador
ajuste.fit(X_b, y_b)

#vamos a consultar los mejores parámetros encontrados
print(ajuste.best_params_)

## El modelo _mantiene_ los mejores parámetros

**NO** es necesario apuntar en una hoja de papel los mejores parámetros. El objeto que produce GridSearchCV ya permite aplicar el método predict() con los mejores parámetros encontrados.

In [ ]:
y_pred = ajuste.predict(X_b)
print("Informe completo con búsqueda exhaustiva de parámetros\n",classification_report(y_b, y_pred))

# Búsqueda aleatoria de Parámetros

## Búsqueda aleatoria de Parámetros

Aunque **GridSearchCV** ofrece muy buenos resultados, cada valor de cada nuevo parámetro multiplica el número de combinaciones, por lo que puede ser muy lento.

Otra opción es utilizar una búsqueda aleatorizada, muestreando cada posible combinación de una distribución de probabilidad (que puede ser elegida por el usuario) que se especifica para cada parámetro a optimizar.

Esta búsqueda tiene la ventaja de que no aumenta el tiempo de búsqueda si los parámetros extra que se quieren buscar no aportan mejor rendimiento.

## Ejemplo de Búsqueda Aleatoria

In [ ]:
#| code-fold: true
from sklearn.model_selection import RandomizedSearchCV
import scipy as sp

# No necesitamos especificar los parámetros
svc = SVC()

# para 'C' y 'gamma' probamos valores usando una distribución exponencial
parametros = {'C': sp.stats.expon(scale=100), 'gamma': sp.stats.expon(scale=.1),
  'kernel': ['rbf']}

# Genera por defecto solo 10 combinaciones, se modifica con n_iter
ajuste = RandomizedSearchCV(svc, parametros, n_iter=10, verbose=3, random_state=12)

ajuste.fit(X_b, y_b)

print(ajuste.best_params_)

y_pred = ajuste.predict(X_b)
print("Informe completo con búsqueda aleatorizada de parámetros\n",classification_report(y_b, y_pred))

---

Otro ejemplo de distribución de probabilidad para los parámetros C y gamma (del kernel RBF) con una loguniform.


In [ ]:
print("Valores de C: ", sp.stats.loguniform(1e0, 1e3))
parametros = {'C': sp.stats.loguniform(1e0, 1e3),
 'gamma': sp.stats.loguniform(1e-4, 1e-3),
 'kernel': ['rbf']}

ajuste = RandomizedSearchCV(svc, parametros)

ajuste.fit(X_b, y_b)

print(ajuste.best_params_)

y_pred = ajuste.predict(X_b)
print("Informe completo con búsqueda aleatorizada de parámetros\n",classification_report(y_b, y_pred))

# Búsqueda de parámetros con *Succesive Halving*

## Búsqueda de parámetros con *Successive halving*

La división sucesiva a la mitad (Successive Halving -SH-) es similar a un carrera entre combinaciones de parámetros candidatos. SH es un proceso de selección iterativo en el que todos los candidatos (las combinaciones de parámetros) se evalúan con una pequeña cantidad de recursos en la primera iteración. Sólo algunos de estos candidatos se seleccionan para la siguiente iteración, a la que se asignan más recursos.

Está disponible tanto para la búsqueda exhaustiva como para la aleatorizada.

## Ejemplo de uso

In [ ]:
#| code-fold: true
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

svc = SVC() 
parametros = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

ajuste = HalvingGridSearchCV(svc, parametros) #la forma de construir la versión SH del grid search es idéntica a éste

ajuste.fit(X_b, y_b)

print(ajuste.best_params_)

y_pred = ajuste.predict(X_b)
print("Informe completo con búsqueda exhaustiva de parámetros (SH)\n",classification_report(y_b, y_pred))

parametros = {'C': sp.stats.expon(scale=100), 'gamma': sp.stats.expon(scale=.1),
  'kernel': ['rbf']}

ajuste = HalvingRandomSearchCV(svc, parametros)

ajuste.fit(X_b, y_b)

print(ajuste.best_params_)

y_pred = ajuste.predict(X_b)
print("Informe completo con búsqueda aleatorizada de parámetros (SH)\n",classification_report(y_b, y_pred))

# Otras alternativas

## Otras alternativas

Existen distintos programas para optimizar parámetros que se pueden usar de forma sencilla para Scikit-learn.

- Hyperopt: [https://hyperopt.github.io/hyperopt/](https://hyperopt.github.io/hyperopt/).
- Optuna: [https://optuna.readthedocs.io/en/stable/](https://optuna.readthedocs.io/en/stable/).

Veremos solo Hyperopt porque tiene un interfaz más sencillo para Sklearn.

## Hyperopt

- Presenta un interfaz distinto de Sklearn, pero el paquete hpsklearn permite una clase que implementa.

In [ ]:
#| output: false
!pip install hyperopt hpsklearn

Luego se usa la clase **HyperoptEstimator** que es similar a las clases anteriores.

Debe de trabajar con sus propias clases de clasificadores, no directamente las de Scikit-learn.

## Ejemplo de Hyperopt

In [ ]:
#| eval: false
from hpsklearn import HyperoptEstimator, svc
from hyperopt import hp

# Usamos  choice, pero tiene para distribuciones normal, uniform, ....
pkernel = hp.choice('kernel', ['linear', 'rbf'])
pC = hp.choice('C', [1, 10])
ajuste = HyperoptEstimator(classifier=svc("mySVC"), max_evals=50)

ajuste.fit(X_b, y_b)
print(ajuste.best_model())
y_pred = ajuste.predict(y_b)
print("Informe completo con búsqueda de parámetros (Hyperopt)\n",classification_report(y_b, y_pred))